In [3]:
import tensorflow as tf

tf.constant([[1., 2., 3.], [4., 5., 6.]]) #matrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t.shape)
print(t.dtype)

(2, 3)
<dtype: 'float32'>


In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis] #... to get one dimension of an array

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t+10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t) #the @ is for multiplication so basically its t.t'

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [11]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [12]:
t.numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>>

In [13]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [14]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [15]:
#Numpy uses 64 bit and tensor uses 32 bit

In [16]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [17]:
#Tensors are immutable and thus you would need to create variables.
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [18]:
v.assign(2 *v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [19]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [20]:
v[:,2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [21]:
v.scatter_nd_update(indices=[[0,0], [1,2]], updates=[100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

Customising models and training algorithms using the above

In [22]:
#Lets create a loss function, the huber loss
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

You can use the above loss now in the model compile method where you can specify the loss=huber_fn

In [23]:
#model.compile(loss=huber_fn, optimizer="nadam")
#model.fit(X_train, y_train)

Loading a model with the custom needs to have the custom parameters

In [24]:
# model = keras.models.load_model("my_model.h5", custom_objects={"huber_fn": huber_fn})

This can also be solved using the subclass of the keras loss class.

In [25]:
from tensorflow import keras

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

Also, customer initializers and activation functions can also be created.

In [26]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

# Then you can use it like so: layer = keras.layers.Dense(30, activation=my_softplus)

There is also something called precision which can be used in order to get the correct output, this would mean that it is able to track the false positives and true positives and give the correct output on precision. 

You can also create custom layers if needed.

In [27]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

You can also create custom losses and metrics if needed.

Computing Gradients using AutoDiff, which is basically an approximation of the derivative without computing the full derivative

In [28]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

In [29]:
w1, w2 = 5,3
eps = 1e-6
(f(w1+eps, w2) - f(w1,w2))/eps

36.000003007075065

In [30]:
(f(w1, w2+eps) - f(w1,w2))/eps

10.000000003174137

In [31]:
#Using Autodiff
w1,w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print(gradients)

#By default,you can only call tape.gradient once, however if you would like to call it multiple times you can set
# (persistent = True) inside the GradientTape but then call del tape at the end
# Only works with Variables
# You can also stop gradients using the stop_gradient() function when defining w

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


In [32]:
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1 + 1/exp)
    return tf.math.log(exp + 1), my_softplus_gradients

Custom Training Loops

In [33]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

Tensorflow functions and graphs

In [34]:
def cube(x):
    return x ** 3

In [35]:
cube(2)

8

In [36]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [37]:
tf_cube = tf.function(cube)

In [38]:
tf_cube

In [39]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>